In [8]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt


In [4]:
words = open("names.txt",'r').read().splitlines()
print(len(words))
print(words[:8])

32033
['emma', 'olivia', 'ava', 'isabella', 'sophia', 'charlotte', 'mia', 'amelia']


In [5]:
#building vocab mappings
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.']=0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)

In [9]:
#build the dataset
block_size = 3

def build_dataset(words):
    X, Y = [],[]
    for w in words:
        context = [0] * block_size
        for c in w + '.':
            ix = stoi[c]
            Y.append(ix)
            X.append(context)
            context = context[1:] + [ix]
    X = torch.tensor(X)
    Y = torch.tensor(Y)

    return X,Y


import random
random.seed(42)
random.shuffle(words)
n1 = int(0.8*len(words))
n2 = int(0.9*len(words))

Xtr, Ytr = build_dataset(words[:n1])
Xdev, Ydev = build_dataset(words[n1:n2])
Xte,Yte = build_dataset(words[n2:])        





In [ ]:
#helper function

In [10]:
n_embed = 10
n_hidden = 100

g = torch.Generator().manual_seed(2147483647)
C = torch.randn((vocab_size,n_embed), generator=g)

W1 = torch.randn((n_embed*block_size,n_hidden), generator=g) * (5/3)/((n_embed * block_size)**0.5)
b1 = torch.randn(n_hidden, generator=g) * 0.1
W2 = torch.randn((n_hidden,vocab_size),generator=g) * 0.1
b2 = torch.randn(vocab_size, generator=g) * 0.1

bnbias = torch.randn((1,n_hidden), generator=g)
bngain = torch.randn((1,n_hidden), generator=g)

parameters = [C,W1,b1,W2,b2]
print(sum(p.nelement() for p in parameters))
for p in parameters:
    p.requires_grad = True



6097


In [11]:
batch_size = 32
n = batch_size
ix = torch.randint(0,Xtr.shape[0],(batch_size,),generator=g)
Xb,Yb = Xtr[ix], Ytr[ix]



In [ ]:
#forawrd pass
embed = C[Xb]
embedcat = embed.view(embed.shape[0],-1)

hpre = embedcat @ W1 + b1
hpmean = 1/n * hpre.sum(0,keepdims= True)
bndiff = hpre - hpmean
bndiff2 = bndiff ** 2
bnvar = 1/(n-1) * (bndiff2).sum(0,keepdims = True)
bnvar_inv = (bnvar + 1e-5)**-0.5
bnraw = bndiff * bnvar_inv

hpreact = bngain * bnraw + bnbias

h = torch.tanh(hpreact)

logits = h @ W2 + b2

#crossentropyloss
logit_maxes = logits.max(1,keepdims = True)
norm_logits =logits - logit_maxes

counts = norm_logits.exp()
counts_sum = counts.sum(1,keepdims = True)
counts_sum_inv = counts_sum**-1
probs = counts * counts_sum_inv
logprobs = probs.log()
loss = -logprobs[range(n),Yb].mean()

for p in parameters:
    p.grad = None
    

